In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
import warnings; warnings.filterwarnings('ignore')
import tensorflow as tf

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 4)

## Load Datasets

In [4]:
iris_datasets = load_iris()
X_iris, y_iris = iris_datasets.data[:-1, :], iris_datasets.target[:-1]
y_iris = pd.get_dummies(y_iris).values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.33, random_state=90)

---